# Analisis de Integridad de Datos

- En este analisis se quiere ver la distribucion de la base de datos DEIS de Egresos Hospitalarios.
- Estos resultados se quieren contrastar con los datos del tablero DEIS MINSAL y con los datos de GRD.
- Los datos solamente corresponderan al estrato del Torax

In [25]:
import polars as pl
import pandas as pd

import lectura_archivos as fr
import funciones_auxiliares_hito_1 as aux

In [28]:
with pl.StringCache():
    df = pl.scan_csv("input/utf-8/*.csv", dtypes=fr.DICT_VARIABLES, separator=";")
    torax = df.filter(pl.col("ESTABLECIMIENTO_SALUD") == 112103)
    torax_pandas = torax.collect().to_pandas()


## Cantidad de Egresos Hospitalarios

- En este caso, se quiere ver la cantidad de egresos hospitalarios por año

In [33]:
numero_egresos_deis = (torax_pandas.groupby("ANO_EGRESO")['DIAG1'].count())
display(numero_egresos_deis)

ANO_EGRESO
2001    4985
2002    5043
2003    5326
2004    5261
2005    5333
2006    5232
2007    5084
2008    5132
2009    4690
2010    4538
2011    4662
2012    4453
2013    4435
2014    4409
2015    4621
2016    4872
2017    4901
2018    5482
2019    5435
2020    2848
Name: DIAG1, dtype: int64

## Cantidad de Valores Faltantes

In [27]:
torax_pandas = 

PanicException: should not fail: ComputeError(ErrString("unable to merge categorical arrays created under different global string caches (try setting a global string cache)"))

In [24]:
aux.analizar_valores_faltantes(torax_pandas)

## Cantidad de Dias Estada Totales

In [1]:
import pandas as pd

import funciones_auxiliares as aux
import funciones_auxiliares_hito_2 as aux2

# Analisis de Integridad de Datos

En este apartado se quiere saber si los datos de los resumenes de Egresos DEIS, son iguales a
las bases de datos en formato largo (paciente por paciente). Se quieren verificar los siguientes
indicadores:

- Las bases de datos presentan la misma cantidad de datos para cada año
- La distribución de enfermedades por gran causa son iguales entre bases de datos para cada año

# Trata de valores de columnas

La columna 16 (Región de residencia) contiene valores mixtos (ints y strings). Por lo tanto, se
unificarán en valores strings

## Análisis de Cantidad de Datos

Según el resumen DEIS de 2013 - 2018, cada año tiene la siguiente cantidad de datos:

Año|Cantidad de datos|
---|-----------------|
2013|1.676.937|
2014|1.660.150|
2015|1.671.054|
2016|1.637.265|
2017|1.637.150|
2018|1.669.602|

Según el resumen DEIS de 2018 - 2021, los años tienen la siguiente cantidad de datos:

Año|Cantidad de datos|
---|-----------------|
2018|1.669.602|
2019|1.667.180|
2020|1.330.477|
2021|1.467.062|

Ahora, se analizará la cantidad de datos de cada base de datos en formato largo:


# Lectura de Archivos

Todos los archivos son separados por ';', y estan en encoding 'latin-1'. Además, el archivo de
2014 tiene 1 línea que está mal imputada (genera error de lectura). Debido a lo anterior,
la línea fue droppeada. Todos los demás archivos fueron leídos correctamente, sin necesidad de
droppear algún valor.

In [2]:
dfs = aux.leer_anios_egresos()
df_completa = pd.concat(dfs.values())

c:\Users\finanzas\Desktop\Javier Rojas\Javier\trabajo\SSMO\EPH\modulo_gestion\1.analisis_problemas_de_salud\egresos-hospitalarios\funciones_auxiliares.py:53: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[anio] = pd.read_csv(ruta, delimiter=';', encoding='latin-1', on_bad_lines='skip')
c:\Users\finanzas\Desktop\Javier Rojas\Javier\trabajo\SSMO\EPH\modulo_gestion\1.analisis_problemas_de_salud\egresos-hospitalarios\funciones_auxiliares.py:53: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[anio] = pd.read_csv(ruta, delimiter=';', encoding='latin-1', on_bad_lines='skip')
c:\Users\finanzas\Desktop\Javier Rojas\Javier\trabajo\SSMO\EPH\modulo_gestion\1.analisis_problemas_de_salud\egresos-hospitalarios\funciones_auxiliares.py:53: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[anio] = pd.read_csv(ruta, delimiter=';', 

In [47]:
for anio, df in dfs.items():
    print(f"El DataFrame de {anio} tiene {df.shape[0]} datos")


El DataFrame de 2013 tiene 1676936 datos
El DataFrame de 2014 tiene 1660150 datos
El DataFrame de 2015 tiene 1671054 datos
El DataFrame de 2016 tiene 1637265 datos
El DataFrame de 2017 tiene 1637150 datos
El DataFrame de 2018 tiene 1669602 datos
El DataFrame de 2019 tiene 1636508 datos


Por lo tanto, la comparacion de datos es:

Año|Cantidad de datos|Base de Datos|
---|-----------------|-------------|
2013|1.676.937|1.676.936|
2014|1.660.150|1.660.150|
2015|1.671.054|1.671.054|
2016|1.637.265|1.637.265|
2017|1.637.150|1.637.150|
2018|1.669.602|1.669.602|
2019|1.667.180|1.636.508|

O sea, **solamente 2013 y 2019 difieren con la cantidad de datos** mostrados en el resumen de DEIS.

# Análisis de Datos Faltantes

En este apartado se quiere ver la cantidad de valores faltantes en las bases de datos

In [3]:
aux2.analizar_valores_faltantes(df_completa)

,cantidad_na,porcentaje_na
ID_PACIENTE,184299,1.59
ESTABLECIMIENTO_SALUD,0,0.00
GLOSA_ESTABLECIMIENTO_SALUD,0,0.00
PERTENENCIA_ESTABLECIMIENTO_SALUD,0,0.00
SEREMI,7391665,63.78
SERVICIO_DE_SALUD,4197000,36.22
SEXO,3,0.00
FECHA_NACIMIENTO,167,0.00
EDAD_CANT,0,0.00
TIPO_EDAD,0,0.00


KeyboardInterrupt: 